In [5]:
%matplotlib notebook
import time
import sklearn
import pandas as pd
import numpy as np
import category_encoders
from numpy.linalg.linalg import LinAlgError
from collections import Counter
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression as lr
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as confu
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.cluster import KMeans
from itertools import combinations as Cb
from sklearn.tree import ExtraTreeClassifier as ex_tree

In [6]:
# Normalize data
def normalization(training_numpy, testing_numpy):
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy)
    testing_x_scaled = min_max.transform(testing_numpy)
    #training_norm = pd.DataFrame(x_scaled, columns = columns)
    #testing_norm = pd.DataFrame(testting_x_scaled, columns = columns)
    return (x_scaled, testing_x_scaled)

def pca_function(rate, data):
    pca = PCA()
    pca.fit(data)
    for thres_n in xrange(1,len(data)):
        if sum(pca.explained_variance_ratio_[:thres_n])>rate:
            pca_n = thres_n
            break
    
    pca = PCA(n_components=pca_n)
    pca_data = pca.fit_transform(data)
    return (pca_data, pca)

def lda_whole_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


def lda_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


'''
def pca_function(n, data):
    pca = PCA(n_components=n)
    pca_data = pca.fit_transform(data)
    return (pca_data.flatten(), pca)

def lda_function(X, y):
    lda = LDA()
    lda_data = lda.fit_transform(X, y)
    return (lda_data.flatten(), lda)
'''
def replace_value(training_normal_numpy, key, loc):  
    training_normal_numpy[loc][key] = 1
    
def del_zero_column(df_train, df_test):
    new_df_test = pd.DataFrame(np.zeros([df_test.shape[0], df_train.shape[1]]),columns=df_train.columns)
    new_df_test[list(set(df_train.columns) & set(df_test.columns))] = df_test[list(set(df_train.columns) & set(df_test.columns))]
    #df_test = df_test[df_train.columns]
    return (df_train, new_df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]
#def del_zero_column(df_train, df_test):
#    df_train = df_train.loc[:, (df_train != 0).any(axis=0)]
#    df_test = df_test[df_train.columns]
#    return (df_train, df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]

def all_lda(train_combine, label_num, n_cluster = 2):
    train_all_dummy = pd.get_dummies(train_combine)
    #train_all_dummy = category_encoders.OneHotEncoder(cols=train_combine.columns.tolist()).fit_transform(train_combine)
    (train_lda, lda) = lda_whole_function(train_all_dummy, np.array(label_num), n_cluster=n_cluster)
    clf=SVC(kernel='linear', C=1.0)
    start = time.time()
    
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score , time.time()-start)

def divide_lda(train_combine, label_num, n_cluster = 2):
    train_column= train_combine.columns
    for col in train_combine.columns:
        train_dummy.append(pd.get_dummies(train_combine[col]))
        #train_dummy.append(category_encoders.OneHotEncoder(cols=[col]).fit_transform(train_combine[[col]]))
    train_lda = pd.DataFrame(index=range(len(label_num)))
    for i in range(len(train_combine.columns)):
        #print train_dummy[i].shape
        (now_train_lda, lda) = lda_whole_function(train_dummy[i], np.array(label_num), n_cluster=3)
        now_lda_col_name = []
        for j in range(len(now_train_lda[0])):
            train_lda[train_column[i]+'_'+str(j+1)] = np.array(now_train_lda)[:,j]
    start = time.time()
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score, time.time()-start)

def combine_selection(train_dummy, label):
    columns = train_dummy.columns
    feature_dic = {}
    new_column = []
    initial = 1
    whole_train_dummy = pd.DataFrame()
    for column, idx in zip(columns, range(len(columns))):     
        now_column = []
        each_train_dummy = pd.get_dummies(train_dummy[column])
        
        clf = ex_tree()
        clf.fit(each_train_dummy, label)
        
        for key, value in zip(each_train_dummy.columns, clf.feature_importances_):
            now_column.append(('combine%d_'+key) % idx)
            new_column.append(('combine%d_'+key) % idx)
            feature_dic[('combine%d_'+key) % idx] = value
        each_train_dummy.columns = now_column
        if initial == 1:
            whole_train_dummy = whole_train_dummy.append(each_train_dummy)
            initial = 0
        else:
            whole_train_dummy = whole_train_dummy.join(each_train_dummy)
        
    #print len(new_column)
    sorted_dic = sorted(feature_dic.iteritems(), key=lambda (k,v): (v,k),reverse=True)   

    return (sorted_dic, whole_train_dummy) 

# DNA

In [9]:
train_dummy.iloc[train_val_index].shape

(2870, 30900)

In [7]:
train_lda = lda.fit_transform(train_dummy.iloc[train_val_index], val_label[train_val_index])

LinAlgError: SVD did not converge

In [36]:
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['combine_num','cluster','top','score', 'time'])
total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
exp_num =0 
for combine_num in range(2,3):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        #print val_label[train_val_index]
        train_label_list.append(val_label[train_val_index])
        #print train_label_list
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(20,1020,20):
         
        val_time = []
        now_score = []
        for cluster in xrange(2,5):
            
            for (cv,(train_val_index,_)) in zip(xrange(10),skf):   
                lda = LDA()
                sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
                train_dummy_top = train_dummy_list[cv][sorted_feature]
                test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
                #print "cv:",cv,"shape:",train_dummy_top.shape

                cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
                test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
                #print sorted_dic
                
                try: 
                    
                    (train_lda,lda) = lda_whole_function(train_dummy_top, train_label_list[cv].copy(), n_cluster=cluster)        
                    test_lda = lda.transform(test_dummy_top)
                    (train_lda, test_lda) = normalization(train_lda, test_lda)                    
                    clf.fit(train_lda, val_label[train_val_index])
                    
                    start = time.time()
                    now_score.append(accuracy_score(clf.predict(test_lda), test_label_list[cv]))
                    val_time.append(time.time() - start)   

                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            exp_num = exp_num + 1
            #total_score.append(np.average(now_score))
            #total_time.append(exe_time)

            print "top: ", threshold, "cluster:", cluster, "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
            result.loc[exp_num] = [combine_num, cluster, threshold, np.average(now_score), exe_time]
            result.to_csv('dna_combine_lda_kmeans.csv')

combine_subset: 1770
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  20 cluster: 2 combine_num 2 score: 0.925994204307 time: 0.0382032394409
top:  20 cluster: 3 combine_num 2 score: 0.927092381156 time: 0.076144695282
top:  20 cluster: 4 combine_num 2 score: 0.926724011695 time: 0.115865468979
top:  40 cluster: 2 combine_num 2 score: 0.936357891834 time: 0.0293562412262
top:  40 cluster: 3 combine_num 2 score: 0.937293910047 time: 0.0637180805206
top:  40 cluster: 4 combine_num 2 score: 0.937500114691 time: 0.101910114288
top:  60 cluster: 2 combine_num 2 score: 0.941995688788 time: 0.0284459590912
top:  60 cluster: 3 combine_num 2 score: 0.943719355449 time: 0.0610883235931
top:  60 cluster: 4 combine_num 2 score: 0.943874623752 time: 0.0979795455933
top:  80 cluster: 2 combine_num 2 score: 0.94076344559 time: 0.0275967121124
top:  80 cluster: 3 combine_num 2 score: 0.942645829852 time: 0.0594639778137
top:  80 cluster: 4 combine_num 2 

# Mushroom

In [35]:
mush_columns = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
               'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',\
               'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type',\
               'spore-print-color', 'population', 'habitat']
train = pd.read_csv('agaricus-lepiota.data',header=None)
train.columns = mush_columns
key_str = {'e':0, 'p':1}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','cluster','top','score', 'time'])
total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
exp_num =0 
for combine_num in range(2,3):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        #print val_label[train_val_index]
        train_label_list.append(val_label[train_val_index])
        #print train_label_list
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(20,1020,20):
         
        val_time = []
        now_score = []
        for cluster in xrange(2,5):
            
            for (cv,(train_val_index,_)) in zip(xrange(10),skf):   
                lda = LDA()
                sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
                train_dummy_top = train_dummy_list[cv][sorted_feature]
                test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
                #print "cv:",cv,"shape:",train_dummy_top.shape

                cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
                test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
                #print sorted_dic
                
                try: 
                    
                    (train_lda,lda) = lda_whole_function(train_dummy_top, train_label_list[cv].copy(), n_cluster=cluster)        
                    test_lda = lda.transform(test_dummy_top)
                    (train_lda, test_lda) = normalization(train_lda, test_lda)                    
                    clf.fit(train_lda, val_label[train_val_index])
                    
                    start = time.time()
                    now_score.append(accuracy_score(clf.predict(test_lda), test_label_list[cv]))
                    val_time.append(time.time() - start)   

                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            exp_num = exp_num + 1
            #total_score.append(np.average(now_score))
            #total_time.append(exe_time)

            print "top: ", threshold, "cluster:", cluster, "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
            result.loc[exp_num] = [combine_num, cluster, threshold, np.average(now_score), exe_time]
            result.to_csv('mush_combine_lda_kmeans.csv')

combine_subset: 231
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  20 cluster: 2 combine_num 2 score: 0.856121320956 time: 0.117446184158
top:  20 cluster: 3 combine_num 2 score: 0.860549365237 time: 0.248730182648
top:  20 cluster: 4 combine_num 2 score: 0.86206627942 time: 0.393080234528
top:  40 cluster: 2 combine_num 2 score: 0.954452690213 time: 0.0555469989777
top:  40 cluster: 3 combine_num 2 score: 0.957464248689 time: 0.122264385223
top:  40 cluster: 4 combine_num 2 score: 0.956417120399 time: 0.187920093536
top:  60 cluster: 2 combine_num 2 score: 0.992853492037 time: 0.0214290618896
top:  60 cluster: 3 combine_num 2 score: 0.973376148775 time: 0.0482468605042
top:  60 cluster: 4 combine_num 2 score: 0.958524420078 time: 0.081289768219
top:  80 cluster: 2 combine_num 2 score: 0.968238989418 time: 0.0186388492584
top:  80 cluster: 3 combine_num 2 score: 0.963990442001 time: 0.0440120697021
top:  80 cluster: 4 combine_num 2 scor

# Car

In [34]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','cluster','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        #print val_label[train_val_index]
        train_label_list.append(val_label[train_val_index])
        #print train_label_list
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(20,1020,20):
         
        val_time = []
        now_score = []
        for cluster in xrange(2,5):
            
            for (cv,(train_val_index,_)) in zip(xrange(10),skf):   
                lda = LDA()
                sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
                train_dummy_top = train_dummy_list[cv][sorted_feature]
                test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)
                #print "cv:",cv,"shape:",train_dummy_top.shape
                cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
                test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
                #print sorted_dic
                
                try:                 
                    (train_lda,lda) = lda_whole_function(train_dummy_top, train_label_list[cv].copy(), n_cluster=cluster)        
                    test_lda = lda.transform(test_dummy_top)
                    (train_lda, test_lda) = normalization(train_lda, test_lda)                    
                    clf.fit(train_lda, val_label[train_val_index])
                    
                    start = time.time()
                    now_score.append(accuracy_score(clf.predict(test_lda), test_label_list[cv]))
                    val_time.append(time.time() - start)   

                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            exp_num = exp_num + 1
            #total_score.append(np.average(now_score))
            #total_time.append(exe_time)

            print "top: ", threshold, "cluster:", cluster, "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
            result.loc[exp_num] = [combine_num, cluster, threshold, np.average(now_score), exe_time]
            result.to_csv('car_combine_lda_kmeans.csv')

combine_subset: 15
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  20 cluster: 2 combine_num 2 score: 0.641919117188 time: 0.0239245891571
top:  20 cluster: 3 combine_num 2 score: 0.62345150011 time: 0.0484127998352
top:  20 cluster: 4 combine_num 2 score: 0.620164657672 time: 0.0756452083588
top:  40 cluster: 2 combine_num 2 score: 0.612780135869 time: 0.019914150238
top:  40 cluster: 3 combine_num 2 score: 0.609535741135 time: 0.0424544811249
top:  40 cluster: 4 combine_num 2 score: 0.608634579146 time: 0.0681967735291
top:  60 cluster: 2 combine_num 2 score: 0.736657562032 time: 0.0170338153839
top:  60 cluster: 3 combine_num 2 score: 0.737307113044 time: 0.0375316143036
top:  60 cluster: 4 combine_num 2 score: 0.735440866709 time: 0.0603020191193
top:  80 cluster: 2 combine_num 2 score: 0.777972233587 time: 0.0153923034668
top:  80 cluster: 3 combine_num 2 score: 0.77012662049 time: 0.0341017246246
top:  80 cluster: 4 combine_num 2 s

In [22]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['combine_num','c_cluster','score', 'time'])


total_score = []
total_time = []
clf = BernoulliNB()
#clf = SVC(kernel='linear', C=1.0)

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
exp_num =0 


for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine_%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    train_dummy = pd.get_dummies(train_combine)
    for cluster in range(2,3):
        pipe = make_pipeline(LDA(), clf)
        #pipe = make_pipeline(LDA(), SVC(kernel='linear', C=1.0))
        #pipe = pipeline([('pca', PCA()),('tree', DecisionTreeClassifier())])
        total_score = []
        total_time = []
        for iteration in range(5):
            val_time = []
            now_score = []
            start = time.time()
            score = cross_val_score(pipe, train_dummy, label_num, cv=10)
            #(score, exe_time) = all_lda(train_combine, label_num, cluster)       
            total_score.append(score)
            total_time.append(time.time() - start)
            
            
        print "combine_num", combine_num
        print "n_cluster:", cluster
        print "score:", np.average(total_score)
        print "time:", np.average(total_time)
        result.loc[exp_num] = [combine_num, cluster, np.average(total_score), np.average(total_time)]
        exp_num = exp_num + 1
        #result.to_csv('car_combine_lda_kmeans_naive.csv')

combine_subset: 15
combine_num 2
n_cluster: 2
score: 0.859080737081
time: 0.569122409821
combine_num 2
n_cluster: 3
score: 0.859080737081
time: 0.520112514496
combine_subset: 20
combine_num 3
n_cluster: 2
score: 0.842033506586
time: 5.34647855759
combine_num 3
n_cluster: 3
score: 0.842033506586
time: 5.33627433777
combine_subset: 15
combine_num 4
n_cluster: 2
score: 0.832286525284
time: 23.0885281563
combine_num 4
n_cluster: 3
score: 0.832286525284
time: 22.9524446011


# Bot

In [61]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_5.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','cluster','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        #print val_label[train_val_index]
        train_label_list.append(val_label[train_val_index])
        #print train_label_list
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
         
        val_time = []
        now_score = []
        for cluster in xrange(2,5):
            
            for (cv,(train_val_index,test_val_index)) in zip(xrange(10),skf):   
                lda = LDA()
                sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
                train_dummy_top = train_dummy_list[cv][sorted_feature]
                test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)


                cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
                test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
                #print sorted_dic
                try:
                    #train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv])
                    (train_lda,lda) = lda_whole_function(train_dummy_top, train_label_list[cv].copy(), n_cluster=cluster)        
                    train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                    test_lda = lda.transform(test_dummy_top)
                    test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                    (train_lda, test_lda) = normalization(train_lda, test_lda)             
                    clf.fit(train_lda, train_label_list[cv])
                    
                    start = time.time()
                    now_score.append(f1_score(clf.predict(test_lda), test_label_list[cv]))
                    val_time.append(time.time() - start)
                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            exp_num = exp_num + 1
            #total_score.append(np.average(now_score))
            #total_time.append(exe_time)

            print "top: ", threshold, "cluster:", cluster, "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
            result.loc[exp_num] = [combine_num, cluster, threshold, np.average(now_score), exe_time]
            result.to_csv('bot5_combine_lda_kmeans.csv')

combine_subset: 6
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2 cluster: 2 combine_num 2 score: 0.0950819672131 time: 0.160819530487
top:  2 cluster: 3 combine_num 2 score: 0.17800538193 time: 0.304108381271
top:  2 cluster: 4 combine_num 2 score: 0.205646520169 time: 0.437315940857
top:  4 cluster: 2 combine_num 2 score: 0.314478674264 time: 0.132560014725
top:  4 cluster: 3 combine_num 2 score: 0.172806138752 time: 0.282643556595
top:  4 cluster: 4 combine_num 2 score: 0.227928671136 time: 0.406450033188
top:  6 cluster: 2 combine_num 2 score: 0.522679865016 time: 0.115573167801
top:  6 cluster: 3 combine_num 2 score: 0.449029582986 time: 0.242498159409
top:  6 cluster: 4 combine_num 2 score: 0.419869686271 time: 0.370575428009
top:  8 cluster: 2 combine_num 2 score: 0.732500238993 time: 0.0937533378601
top:  8 cluster: 3 combine_num 2 score: 0.717264597074 time: 0.201056718826
top:  8 cluster: 4 combine_num 2 score: 0.657167350629

In [62]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_6.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','cluster','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        #print val_label[train_val_index]
        train_label_list.append(val_label[train_val_index])
        #print train_label_list
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
         
        val_time = []
        now_score = []
        for cluster in xrange(2,5):
            
            for (cv,(train_val_index,test_val_index)) in zip(xrange(10),skf):   
                lda = LDA()
                sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
                train_dummy_top = train_dummy_list[cv][sorted_feature]
                test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)


                cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
                test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
                #print sorted_dic
                try:
                    #train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv])
                    (train_lda,lda) = lda_whole_function(train_dummy_top, train_label_list[cv].copy(), n_cluster=cluster)        
                    train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                    test_lda = lda.transform(test_dummy_top)
                    test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                    (train_lda, test_lda) = normalization(train_lda, test_lda)             
                    clf.fit(train_lda, train_label_list[cv])
                    
                    start = time.time()
                    now_score.append(f1_score(clf.predict(test_lda), test_label_list[cv]))
                    val_time.append(time.time() - start)
                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            exp_num = exp_num + 1
            #total_score.append(np.average(now_score))
            #total_time.append(exe_time)

            print "top: ", threshold, "cluster:", cluster, "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
            result.loc[exp_num] = [combine_num, cluster, threshold, np.average(now_score), exe_time]
            result.to_csv('bot6_combine_lda_kmeans.csv')

combine_subset: 6
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2 cluster: 2 combine_num 2 score: 0.922147702128 time: 0.359313249588
top:  2 cluster: 3 combine_num 2 score: 0.94822185015 time: 0.464504003525
top:  2 cluster: 4 combine_num 2 score: 0.956913232824 time: 0.569476604462
top:  4 cluster: 2 combine_num 2 score: 0.938824996558 time: 0.281516313553
top:  4 cluster: 3 combine_num 2 score: 0.948408264716 time: 0.477936506271
top:  4 cluster: 4 combine_num 2 score: 0.95703579324 time: 0.583919286728
top:  6 cluster: 2 combine_num 2 score: 0.957996680758 time: 0.20120549202
top:  6 cluster: 3 combine_num 2 score: 0.957996680758 time: 0.392370700836
top:  6 cluster: 4 combine_num 2 score: 0.963465716091 time: 0.497911930084
top:  8 cluster: 2 combine_num 2 score: 0.978741770963 time: 0.0993378162384
top:  8 cluster: 3 combine_num 2 score: 0.978741770963 time: 0.196998119354
top:  8 cluster: 4 combine_num 2 score: 0.977259846699 ti

In [63]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_7.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['combine_num','cluster','top','score', 'time'])



total_score = []
total_time = []
clf = SVC(kernel='linear', C=1.0)
#clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)

exp_num = 0
for combine_num in range(2,5):
    
    train_combine = pd.DataFrame(index=range(len(train)))
    com_index = 0
    for sub in Cb(train.columns,combine_num):
        combine_list = []
        map(lambda data: combine_list.append(combine_function(data)),train[list(sub)].values)
        train_combine['combine%d' % com_index] = combine_list
        com_index = com_index + 1
    print "combine_subset:", com_index
    #train_dummy_top = combine_selection(train_dummy)
    #train_dummy = pd.get_dummies(train_combine)
    total_score = []
    total_time = []
    
    train_dummy_list = []
    sorted_dic_list = []
    test_dummy_list = []
    train_label_list = []
    test_label_list = []
    
    rd = 0
    for train_val_index, test_val_index in skf:
        print "round:", rd+1
        rd = rd+1
        (sorted_dic, whole_train_dummy) = combine_selection(train_combine.iloc[train_val_index],val_label[train_val_index])
        test_dummy = pd.get_dummies(train_combine.iloc[test_val_index])
        train_dummy_list.append(whole_train_dummy)
        sorted_dic_list.append(sorted_dic)
        test_dummy_list.append(test_dummy)
        #print val_label[train_val_index]
        train_label_list.append(val_label[train_val_index])
        #print train_label_list
        test_label_list.append(val_label[test_val_index])
        
    for threshold in xrange(2,test_dummy_list[0].shape[1],2):
         
        val_time = []
        now_score = []
        for cluster in xrange(2,5):
            
            for (cv,(train_val_index,test_val_index)) in zip(xrange(10),skf):   
                lda = LDA()
                sorted_feature = [i[0] for i in sorted_dic_list[cv]][:threshold]
                train_dummy_top = train_dummy_list[cv][sorted_feature]
                test_dummy_top = pd.DataFrame(np.zeros([test_dummy_list[cv].shape[0], len(sorted_feature)]),columns=sorted_feature, index=test_dummy_list[cv].index)


                cross_column = list(set(sorted_feature) & set(test_dummy_list[cv].columns))
                test_dummy_top[cross_column] = test_dummy_list[cv][cross_column]
                #print sorted_dic
                try:
                    #train_lda = lda.fit_transform(train_dummy_top, train_label_list[cv])
                    (train_lda,lda) = lda_whole_function(train_dummy_top, train_label_list[cv].copy(), n_cluster=cluster)        
                    train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
                    test_lda = lda.transform(test_dummy_top)
                    test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
                    (train_lda, test_lda) = normalization(train_lda, test_lda)             
                    clf.fit(train_lda, train_label_list[cv])
                    
                    start = time.time()
                    now_score.append(f1_score(clf.predict(test_lda), test_label_list[cv]))
                    val_time.append(time.time() - start)
                except LinAlgError:
                    None
            exe_time = np.sum(val_time)
            exp_num = exp_num + 1
            #total_score.append(np.average(now_score))
            #total_time.append(exe_time)

            print "top: ", threshold, "cluster:", cluster, "combine_num", combine_num, "score:", np.average(now_score), "time:", exe_time
            result.loc[exp_num] = [combine_num, cluster, threshold, np.average(now_score), exe_time]
            result.to_csv('bot7_combine_lda_kmeans.csv')

combine_subset: 6
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
top:  2 cluster: 2 combine_num 2 score: 0.265353535354 time: 0.015123128891
top:  2 cluster: 3 combine_num 2 score: 0.298686868687 time: 0.0292689800262
top:  2 cluster: 4 combine_num 2 score: 0.309797979798 time: 0.0456967353821
top:  4 cluster: 2 combine_num 2 score: 0.494163059163 time: 0.017688035965
top:  4 cluster: 3 combine_num 2 score: 0.481663059163 time: 0.0314078330994
top:  4 cluster: 4 combine_num 2 score: 0.469163059163 time: 0.0450851917267
top:  6 cluster: 2 combine_num 2 score: 0.481385281385 time: 0.0169188976288
top:  6 cluster: 3 combine_num 2 score: 0.470079365079 time: 0.0304658412933
top:  6 cluster: 4 combine_num 2 score: 0.404353054353 time: 0.044597864151
top:  8 cluster: 2 combine_num 2 score: 0.494163059163 time: 0.0135521888733
top:  8 cluster: 3 combine_num 2 score: 0.476468253968 time: 0.0254592895508
top:  8 cluster: 4 combine_num 2 score: 0.37623

In [6]:
train_lda.shape

(1567, 10)